In [1]:
pip install git+https://github.com/salaniz/pycocoevalcap

  Cloning https://github.com/salaniz/pycocoevalcap to /tmp/pip-req-build-hxgj1mh2
  Running command git clone --filter=blob:none --quiet https://github.com/salaniz/pycocoevalcap /tmp/pip-req-build-hxgj1mh2
  Resolved https://github.com/salaniz/pycocoevalcap to commit a24f74c408c918f1f4ec34e9514bc8a76ce41ffd
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 2.7 MB/s eta 0:00:00
  Created wheel for pycocoevalcap: filename=pycocoevalcap-1.2-py3-none-any.whl size=104312245 sha256=56f8401679573459b207bdc8570de334f5485cec9453b6e124f87b059c67633a
  Stored in directory: /tmp/pip-ephem-wheel-cache-zfse64t1/wheels/43/54/73/3e2c6d4ace7657958cde52ac6fd47b342cd4aae5a7aa4fcbf9
Successfully built pycocoevalcap
Note: you may need to restart the kernel to use updated packages.


# Imports required

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torchvision import models
import os
import pandas as pd
import spacy
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim
import statistics
from tqdm import tqdm
import pycocoevalcap
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from torchmetrics.text import BLEUScore
from transformers import ViTModel, ViTImageProcessor
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")

2024-04-17 03:35:43.627453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-17 03:35:43.627555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-17 03:35:43.898527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Preprocessing

The below cell consists of: 
- <b>Vocabulary class</b>: This is required to build a vocabulary which consists of words which are repeated atleast 5 times(frequency threshold) and four new tokens(<SOS>(Start of Sentence), <PAD>(Padding token to make consistent length), <EOS> (End of Sentence), <UNK> (Token to print any Unknown word))
- <b>ImgCapDataset</b>: This class inherits from Pytorch Dataset which is required to build a dataset donsisting of preprocessed images, captions to use them for model training.
- <b>mycoll</b>: This class is used to provide the required elements for training from a batch.
- <b>get_loamder</b>: This function is used to get the dataloader using DataLoader from pytorch and uses our custom collate class above to return the dataset in batches of 32.

In [3]:
spacy_eng = spacy.load("en_core_web_sm")

class vocabularry:
    def __init__(self, fr_th):
        self.int_to_str = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.str_to_int = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.fr_th = fr_th

    def __len__(self):
        return len(self.int_to_str)

    @staticmethod
    def tok_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabularry(self, sen_lst):
        frequ = {}
        idx = 4

        for sent in sen_lst:
            for word in self.tok_eng(sent):
                if word not in frequ:
                    frequ[word] = 1

                else:
                    frequ[word] += 1

                if frequ[word] == self.fr_th:
                    self.str_to_int[word] = idx
                    self.int_to_str[idx] = word
                    idx += 1

    def numriclise(self, text):
        tokenized_text = self.tok_eng(text)

        return [
            self.str_to_int[token] if token in self.str_to_int else self.str_to_int["<UNK>"]
            for token in tokenized_text
        ]


class ImgCapDataset(Dataset):
    def __init__(self, root_dir, cap_file, trans=None, fr_th=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(cap_file)
        self.trans = trans
        self.imgs = self.df["filename"]
        self.caps = self.df["caption"]
        self.vocab = vocabularry(fr_th)
        self.vocab.build_vocabularry(self.caps.tolist())
        self.processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        cap = self.caps[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        if self.trans is not None:
            img = self.trans(img)
            
        processed = self.processor(images = img, return_tensors =  "pt", do_rescale = False)
        img = processed["pixel_values"].squeeze(0)
        numriclised_cap = [self.vocab.str_to_int["<SOS>"]]
        numriclised_cap += self.vocab.numriclise(cap)
        numriclised_cap.append(self.vocab.str_to_int["<EOS>"])

        return img, torch.tensor(numriclised_cap), cap, img_id


class mycoll:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)
        caps = [item[2] for item in batch]
        img_ids = [item[3] for item in batch]
        return imgs, targets, caps, img_ids


def get_loamder(
    root_folder,
    annot_file,
    trans,
    batch_size=32,
    num_workers=4,
    shuffle=True,
    pin_memory=True,
):
    dataset = ImgCapDataset(root_folder, annot_file, trans=trans)

    pad_idx = dataset.vocab.str_to_int["<PAD>"]

    loamder = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=mycoll(pad_idx=pad_idx),
    )

    return loamder, dataset

# Decoder, Encoder and Model

The below cell consists of: 
- <b>image_encoder</b>: This is the encoder class which inherits from torch.nn.Module and takes preprocessed images as inputs for pretrained Vision Transformer. An embedding layer is used to map the <b>class token</b> output of Vision Transformer to the decoder embedding size along with a ReLU function and a dropout layer.
- <b>cap_decoder</b>: This class inherits from torch.nn.Module and uses <b>LSTM</b> as decoder to take inputs or image_features from the encoder and decode them to captions.
- <b>Image_Captioner</b>: This class combines both encoder and decoder defined above to make a collective model.
- <b>sve_ckpt</b>: This function is used to save the model and optimizer parameters to a .pth file.

In [4]:
class image_encoder(nn.Module):
    def __init__(self, emb_size):
        super(image_encoder, self).__init__()
        self.vit = ViTModel.from_pretrained('google/vit-base-patch16-224')
        
        # replace the classifier with a fully connected embedding layer
        self.embed = torch.nn.Linear(self.vit.config.hidden_size, emb_size)

        self.RELUU = nn.ReLU()
        self.dr = nn.Dropout(0.5)

    def forward(self, imgs):
        out = self.vit(pixel_values = imgs, return_dict = True)
        outputs = out.last_hidden_state[:, 0, :]
        feat_encoderr = self.embed(outputs)
        return self.dr(self.RELUU(feat_encoderr))


class cap_decoder(nn.Module):
    def __init__(self, emb_size, hid_size, voc_size, num_layers):
        super(cap_decoder, self).__init__()
        self.emb = nn.Embedding(voc_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hid_size, num_layers)
        self.linear = nn.Linear(hid_size, voc_size)
        self.dr = nn.Dropout(0.5)

    def forward(self, feat_encoderr, captions):
        embeddings = self.dr(self.emb(captions))
        embeddings = torch.cat((feat_encoderr.unsqueeze(0), embeddings), dim=0)
        hid, _ = self.lstm(embeddings)
        out = self.linear(hid)
        return out


class Image_Captioner(nn.Module):
    def __init__(self, emb_size, hid_size, voc_size, num_layers):
        super(Image_Captioner, self).__init__()
        self.cap_encoder = image_encoder(emb_size)
        self.cap_decoder = cap_decoder(emb_size, hid_size, voc_size, num_layers)

    def forward(self, imgs, captions):
        feat_encoderr = self.cap_encoder(imgs)
        out = self.cap_decoder(feat_encoderr, captions)
        return out

    def cap_img(self, image, vocab, m_len=50):
        res_cap = []

        with torch.no_grad():
            x = self.cap_encoder(image).unsqueeze(0)
            sts = None

            for _ in range(m_len):
                hid, sts = self.cap_decoder.lstm(x, sts)
                out = self.cap_decoder.linear(hid.squeeze(0))
                predi = out.argmax(1)
                res_cap.append(predi.item())
                x = self.cap_decoder.emb(predi).unsqueeze(0)

                if vocab.int_to_str[predi.item()] == "<EOS>":
                    break

        return [vocab.int_to_str[idx] for idx in res_cap]
    
def sve_ckpt(st, filename):
    torch.save(st, filename)

# Training the model 

Here we train the last layer and pooler layer of Vision Transformer along with embedding layer of the encoder. We also train the LSTM decoder and evaluate the cider, blue, rouge-l metrics of the validation data.

In [5]:
warnings.filterwarnings("ignore")
def comp_cider(ref, hyp):
    scorer = Cider()
    score, _ = scorer.compute_score(ref, hyp)
    return score
def comp_bleu(ref, hyp):
    scorer = BLEUScore(n_gram=1)
    score = scorer(ref, hyp)
    return score.numpy()
def comp_rougel(ref, hyp):
    scorer = Rouge()
    score, _ = scorer.compute_score(ref, hyp)
    return score
train_transform = transforms.Compose(
    [
        transforms.Resize((300, 300)),
        transforms.RandomCrop((224, 224)),
        transforms.ToTensor(),
    ]
)
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)
train_loamder, dataset = get_loamder(
    root_folder="/kaggle/input/image-captioning/custom_captions_dataset/train",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/train.csv",
    trans=train_transform,
    num_workers=2,
)

val_loamder, val_dataset = get_loamder(root_folder="/kaggle/input/image-captioning/custom_captions_dataset/val",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/val.csv",
    trans=transform,batch_size=1,shuffle=False,
    num_workers=2,)

test_loamder, test_dataset = get_loamder(root_folder="/kaggle/input/image-captioning/custom_captions_dataset/test",
    annot_file="/kaggle/input/image-captioning/custom_captions_dataset/test.csv",
    trans=transform,batch_size=1,shuffle=False,
    num_workers=2,)
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
save_model = False

emb_size = 512
hid_size = 512
voc_size = len(dataset.vocab)
num_layers = 2
l_r = 3e-4
n_epoc = 300

step = 0

model = Image_Captioner(emb_size, hid_size, voc_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.str_to_int["<PAD>"])
optimiz = optim.Adam(model.parameters(), lr=l_r)

for name, params in model.cap_encoder.vit.named_parameters():
    if "vit.pooler.dense.bias" in name or "vit.pooler.dense.bias" in name or "vit.encoder.layer.11" in name:
        params.requires_grad = True
    else:
        params.requires_grad = False


for epoch in range(n_epoc):
    print(f"Epoch: {epoch+1}/{n_epoc}")
    model.train()

    for idx, (imgs, captions, str_cap, _) in tqdm(
        enumerate(train_loamder), total=len(train_loamder), leave=False
    ):
        imgs = imgs.to(device)
        captions = captions.to(device)

        out = model(imgs, captions[:-1])
        loss = criterion(
            out.reshape(-1, out.shape[2]), captions.reshape(-1)
        )

        step += 1

        optimiz.zero_grad()
        loss.backward(loss)
        optimiz.step()
    ref = {}
    hyp = {}
    bleu_ = []
    for idx, (img, captions, str_cap, _) in enumerate(val_loamder):
        img = img
        ref[idx] = str_cap
        a = model.cap_img(img.to(device), dataset.vocab)
        s = ""
        s += a[1]
        for i in range(2,len(a)):
            if a[i] == '.':
                s += a[i]
            elif a[i] == "<EOS>":
                continue
            else:
                s += " "
                s += a[i]
        hyp[idx] = [s]
        bleu_.append(comp_bleu([s],[str_cap]))
    print("CIDEr score of val set: ", comp_cider(ref,hyp))
    print("BLEU score of val set: ", sum(bleu_)/len(bleu_))
    print("ROUGE-l score of val set: ", comp_rougel(ref,hyp))

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1/300


...

CIDEr score of val set:  0.06196359270051961
BLEU score of val set:  0.22106398706251545
ROUGE-l score of val set:  0.20468146752847965
Epoch: 297/300


CIDEr score of val set:  0.061775747401024976
BLEU score of val set:  0.22472219371057683
ROUGE-l score of val set:  0.20458975413546482
Epoch: 298/300


CIDEr score of val set:  0.06016187281221636
BLEU score of val set:  0.22391255864042622
ROUGE-l score of val set:  0.20593438365105235
Epoch: 299/300


CIDEr score of val set:  0.05608151456023881
BLEU score of val set:  0.2253008050295592
ROUGE-l score of val set:  0.20758355350276853
Epoch: 300/300


CIDEr score of val set:  0.05202339282816024
BLEU score of val set:  0.22609440215920387
ROUGE-l score of val set:  0.20469887503689205


# Testing the model on Test Set along with evaluation metrics

In [6]:
warnings.filterwarnings("ignore")
model.eval()
ref = {}
hyp = {}
bleu_ = []
print('-'*50)
for idx, (img, captions, str_cap, img_id) in enumerate(test_loamder):
    img = img
    ref[idx] = str_cap
    a = model.cap_img(img.to(device), dataset.vocab)
    s = ""
    s += a[1]
    for i in range(2,len(a)):
        if a[i] == '.':
            s += a[i]
        elif a[i] == "<EOS>":
            continue
        else:
            s += " "
            s += a[i]
    hyp[idx] = [s]
    bleu_.append(comp_bleu([s],[str_cap]))
    print("Image Name       :", img_id)
    print()
    print("Correct Caption  :  ", ref[idx])
    print()
    print("Predicted Caption: ", [hyp[idx][0]])
    print('-'*50)
print("CIDEr score of test set: ", comp_cider(ref,hyp))
print("BLEU score of test set: ", sum(bleu_)/len(bleu_))
print("ROUGE-l score of test set: ", comp_rougel(ref,hyp))
print()
print()

--------------------------------------------------
Image Name       : ['test_1.jpg']

Correct Caption  :   ['A large building with bars on the windows in front of it. There is people walking in front of the building. There is a street in front of the building with many cars on it. ']

Predicted Caption:  ['a large white building sits behind a street. there is a yellow and green bus on the street. there are people standing in front of the building.']
--------------------------------------------------
Image Name       : ['test_2.jpg']

Correct Caption  :   ['A person is skiing through the snow. There is loose snow all around them from him jumping. The person is wearing a yellow snow suit. The person is holding two ski poles in their hands. ']

Predicted Caption:  ['a person in a yellow , green , and black jacket and black pants is on an orange , purple , and yellow snowboard. the board is in the air as the rider holds it with his right hand. there is a <UNK> of snow below the']
---------